In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import transformers
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

from transformers import RobertaTokenizer, RobertaModel

import sys
sys.path.append('/content/drive/MyDrive/skripsi')
!ls /content/drive/MyDrive/skripsi

 app.py			      finetuning_RoBERTa3.ipynb
 check_lang.py		      finetuning_RoBERTa.ipynb
 cluster_sinta2.ipynb	      functions.py
 cluster_sinta3.ipynb	      model
 cluster_sinta.ipynb	      predict_multibert_kmeans_finetuned.ipynb
'Copy of Untitled0.ipynb'     Preprocessing_data_sinta.ipynb
 data			      Preprocessing.py
 finetuned_multibert2.ipynb   __pycache__
 finetuned_pipeline.py	      src
 finetuning.py		      Untitled0.ipynb
 finetuning_RoBERTa2.ipynb    Untitled1.ipynb


In [ ]:
from Preprocessing import preprocess_text

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/skripsi/data/data_sinta_cleaned3.csv')
df

,jid,desc,title,date,journal,data_cleaned,eissn,jid_umum
0,1,Belakangan ini Lembaga PAUD berbondong-bondong...,Kemampuan Bahasa Inggris Awal pada Periode Lin...,2023-10-05,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,kemampuan bahasa inggris awal pada periode lin...,2549-8959,1
1,1,The assessment process carried out in schools ...,Strengthening Early Childhood Learning Outcome...,2022-06-09,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,strengthening early childhood learning outcome...,2549-8959,1
2,1,Kewajiban guru PAUD untuk memberikan layanan b...,Eksplorasi Deskriptif tentang Layanan Bimbinga...,2023-06-07,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,eksplorasi deskriptif tentang layanan bimbinga...,2549-8959,1
3,1,Pandemi Covid-19 merambah sejak awal tahun 202...,Perspektif Orang Tua terhadap Pelaksanaan Les ...,2021-06-02,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,perspektif orang tua terhadap pelaksanaan les ...,2549-8959,1
4,1,Dalam upaya mencapai Sustainable Development G...,Strategi Pencegahan dan Penanganan Stunting Mu...,2023-12-26,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,strategi pencegahan dan penanganan stunting mu...,2549-8959,1
...,...,...,...,...,...,...,...,...
8670,12,The purpose of this study to examine the effec...,"PENGARUH KINERJA KEUANGAN, DANA ALOKASI UMUM D...",2017-05-15,E-JURNAL AKUNTANSI,pengaruh kinerja keuangan dana alokasi umum da...,2302-8556,12
8671,12,The level of understanding of accounting to be...,"PENGARUH KECERDASAN EMOSIONAL, KECERDASAN INTE...",2017-08-06,E-JURNAL AKUNTANSI,pengaruh kecerdasan emosional kecerdasan intel...,2302-8556,12
8672,12,The purpose of this study was to examine the i...,"PENGARUH UKURAN PERUSAHAAN, KOMPLEKSITAS OPERA...",2017-02-15,E-JURNAL AKUNTANSI,pengaruh ukuran perusahaan kompleksitas operas...,2302-8556,12
8673,12,The purpose of this study is to determine the ...,KINERJA DINAS PARIWISATA BALI BERDASARKAN KONS...,2016-11-10,E-JURNAL AKUNTANSI,kinerja dinas pariwisata bali berdasarkan kons...,2302-8556,12


In [ ]:
df.jid = df.jid.apply(lambda x : x - 1)

In [ ]:
df.dropna(inplace=True)
df = df[df.desc != 'KOSONG']
df = df[df.title != 'KOSONG']
df

,jid,desc,title,date,journal,data_cleaned,eissn,jid_umum
0,0,Belakangan ini Lembaga PAUD berbondong-bondong...,Kemampuan Bahasa Inggris Awal pada Periode Lin...,2023-10-05,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,kemampuan bahasa inggris awal pada periode lin...,2549-8959,1
1,0,The assessment process carried out in schools ...,Strengthening Early Childhood Learning Outcome...,2022-06-09,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,strengthening early childhood learning outcome...,2549-8959,1
2,0,Kewajiban guru PAUD untuk memberikan layanan b...,Eksplorasi Deskriptif tentang Layanan Bimbinga...,2023-06-07,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,eksplorasi deskriptif tentang layanan bimbinga...,2549-8959,1
3,0,Pandemi Covid-19 merambah sejak awal tahun 202...,Perspektif Orang Tua terhadap Pelaksanaan Les ...,2021-06-02,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,perspektif orang tua terhadap pelaksanaan les ...,2549-8959,1
4,0,Dalam upaya mencapai Sustainable Development G...,Strategi Pencegahan dan Penanganan Stunting Mu...,2023-12-26,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,strategi pencegahan dan penanganan stunting mu...,2549-8959,1
...,...,...,...,...,...,...,...,...
8670,11,The purpose of this study to examine the effec...,"PENGARUH KINERJA KEUANGAN, DANA ALOKASI UMUM D...",2017-05-15,E-JURNAL AKUNTANSI,pengaruh kinerja keuangan dana alokasi umum da...,2302-8556,12
8671,11,The level of understanding of accounting to be...,"PENGARUH KECERDASAN EMOSIONAL, KECERDASAN INTE...",2017-08-06,E-JURNAL AKUNTANSI,pengaruh kecerdasan emosional kecerdasan intel...,2302-8556,12
8672,11,The purpose of this study was to examine the i...,"PENGARUH UKURAN PERUSAHAAN, KOMPLEKSITAS OPERA...",2017-02-15,E-JURNAL AKUNTANSI,pengaruh ukuran perusahaan kompleksitas operas...,2302-8556,12
8673,11,The purpose of this study is to determine the ...,KINERJA DINAS PARIWISATA BALI BERDASARKAN KONS...,2016-11-10,E-JURNAL AKUNTANSI,kinerja dinas pariwisata bali berdasarkan kons...,2302-8556,12


In [ ]:
# Define Dataset class
class RobertaDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = 256

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['data_cleaned']
        labels = self.data.iloc[index][['jid']].values.astype(int)
        encoding = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=self.max_length)
        input_ids = encoding['input_ids'][0]
        attention_mask = encoding['attention_mask'][0]
        input_ids = nn.functional.pad(input_ids, (0, self.max_length - input_ids.shape[0]), value=0)
        attention_mask = nn.functional.pad(attention_mask, (0, self.max_length - attention_mask.shape[0]), value=0)
        return input_ids, attention_mask, torch.Tensor(labels)

In [ ]:
# Load tokenizer and model
model_checkpoint = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = RobertaDataset(df, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class RobertaClassifier(nn.Module):
    def __init__(self, num_labels):
        super(RobertaClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained(model_checkpoint)
        self.classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs['last_hidden_state'][:, 0, :]
        x = self.classifier(x)
        return x

In [ ]:
num_labels = 12
model = RobertaClassifier(num_labels).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

num_epochs = 2
n_total_steps = len(train_loader)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for epoch in range(num_epochs):
    for i, batch in enumerate(train_loader):
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.view(-1)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels.long())
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch + 1}/{num_epochs}, batch {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1/2, batch 100/272, loss = 1.9526
epoch 1/2, batch 200/272, loss = 1.4021
epoch 2/2, batch 100/272, loss = 0.9137
epoch 2/2, batch 200/272, loss = 0.7739


In [ ]:
# Create the directory if it doesn't exist
os.makedirs('model', exist_ok=True)
# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/skripsi/model/finetuned_model_roberta3.pt')